Notebook này dùng ds101 là đủ

Dự kiến sẽ viết thành .py

TODO:
- Viết hàm download
- Xử lí Duplicate (có thể dùng pandas)

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib3
import os
import time
import requests

In [3]:
def getSongLinks(country, tab, start_page=1, end_page=20):
    '''
    Trả về danh sách link tới các bài hát cụ thể để nghe nhạc.
    
    Tham số:
        @country: Thể loại nhạc theo tên quốc gia
        @tab: Tab của trang web: album-2021, album-moi, bai-hat-moi, vua-download
        @start_page: Giới hạn bên trái của trang thể hiện bài hát (def=1)
        @end_page: Giới hạn bên phải của trang thể hiện bài hát (def=20)
    
    Lưu ý:
        Với mỗi tab `vua-download` và `bao-hat-moi`
        sẽ chỉ crawl được trang thuộc [1; 19].
        Trang 19+ không crawl được, đừng để bị số lượng trang lừa ¯_(ツ)_/¯
    
    '''
    driver = webdriver.Chrome("chromedriver_win32/chromedriver")
    URL = "https://www.chiasenhac.vn/mp3/" + country + ".html?tab=" + tab
    arr = np.arange(start_page, end_page)
    save_path = 'data/'
    link_list = []

    for i in arr:
        driver.get(URL + "&page=" + str(i))
        main = driver.find_element_by_class_name('content-current')
        songs = main.find_elements_by_css_selector('.media.align-items-stretch.not')
        for song in songs:
            a = song.find_element_by_tag_name("a")
            link = a.get_attribute("href")
            link_list.append(link)

    driver.close()
    return link_list

In [4]:
def getSongInfo(link_list):
    '''
    Trả về danh sách thông tin sau của bài hát theo dạng tuple:
    1. Title
    2. Artist
    3. Link bài hát
    4. Link .m4a
    5. Link .mp3
    '''
    song_info_list = []
    
    for url in link_list:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        # lấy Title và Artist
        name = soup.find(class_="d-flex justify-content-between mb-3 box1 music-listen-title")
        name = name.h1.text.split(" - ")
        title = name[0]
        artist = name[1]

        # lấy link tải m4a và mp3
        download_div = soup.find(id="pills-download")
        download_items = download_div.find_all("a", class_="download_item", href=True)
        m4a_link = download_items[0]['href']
        mp3_link = download_items[1]['href']

        # Thêm bài hát vào danh sách
        song_info_list.append((title, artist, url, m4a_link, mp3_link))
        
    return song_info_list

In [29]:
def writeInfoToCSV(csv_path, song_info_list, remove_duplicate=False):
    '''
    Viết thông tin vào 1 file csv, hàm sẽ tạo file mới nếu file chưa tồn tại.
    
    Tham số:
        @csv_path: Đường dẫn tới file .csv
        @song_info_list: danh sách các tuple info
        @remove_duplicate: xóa các bài hát bị trùng (def=False)
        
    TODO: viết tính năng remove_duplicate
    '''
    # Tạo file .csv nếu chưa tồn tại
    if os.path.exists(csv_path) is not True:
        # Make a new list
        print("Making new file: " + csv_path)
        os.makedirs(os.path.dirname(csv_path), exist_ok=True)
        with open(csv_path, "w") as f:
            f.write("Title,Artist,Link,.m4a,.mp3\n")
        f.close
    else:
        print(csv_path + " already exists, append new lines")
    
    # Viết info vào file .csv
    file = open(csv_path, 'a', encoding="utf-8")
    for i in range(len(song_info_list)):
        title = song_info_list[i][0]
        artist = song_info_list[i][1]
        url = song_info_list[i][2]
        m4a_link = song_info_list[i][3]
        mp3_link = song_info_list[i][4]
        file.write('{},{},{},{},{}\n'.format(title,artist,url,m4a_link,mp3_link))
    file.close()

## Điều chỉnh thông số tại đây

In [30]:
country = "us-uk"
tab = "vua-download"
csv_path = "data/song_info.csv"

In [6]:
# Lấy link
link_list = getSongLinks(country, tab)
print(link_list)

In [ ]:
# Trích xuất info vào: list[tuple]
song_info_list = getSongInfo(link_list)

In [7]:
# Viết file vào trong .csv
writeInfoToCSV(csv_path,song_info_list)